# Logistic Regression with Grid Search (scikit-learn)

<a href="https://colab.research.google.com/github/VertaAI/modeldb/blob/master/client/workflows/demos/sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

This example features:
- **scikit-learn**'s `LinearRegression` model
- **verta**'s Python client logging grid search results
- **verta**'s Python client retrieving the best run from the grid search to calculate full training accuracy
- predictions against a deployed model

In [2]:
HOST = "dev.verta.ai"

PROJECT_NAME = "Census Income Classification"
EXPERIMENT_NAME = "Logistic Regression"

In [3]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 

## Imports

In [4]:
from __future__ import print_function

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import multiprocessing
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [5]:
try:
    import wget
except ImportError:
    !pip install wget  # you may need pip3
    import wget

---

# Log Workflow

## Prepare Data

In [6]:
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.detect_filename(train_data_url)
if not os.path.isfile(train_data_filename):
    wget.download(train_data_url)

test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.detect_filename(test_data_url)
if not os.path.isfile(test_data_filename):
    wget.download(test_data_url)

In [7]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

df_train.head()

,age,capital-gain,capital-loss,hours-per-week,workclass_local-gov,workclass_private,workclass_self-emp-inc,workclass_self-emp-not-inc,workclass_state-gov,workclass_without-pay,...,occupation_handlers-cleaners,occupation_machine-op-inspct,occupation_other-service,occupation_priv-house-serv,occupation_prof-specialty,occupation_protective-serv,occupation_sales,occupation_tech-support,occupation_transport-moving,>50k
0,44,0,0,40,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,0,0,40,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,53,7298,0,60,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,49,0,0,40,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,53,0,1485,40,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


## Prepare Hyperparameters

In [8]:
hyperparam_candidates = {
    'C': [1e-6, 1e-4],
    'solver': ['lbfgs'],
    'max_iter': [15, 28],
}
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

## Instantiate Client

In [9]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

set email from environment
set developer key from environment
connection successfully established
set existing Project: Census Income Classification from personal workspace
set existing Experiment: Logistic Regression


## Train Models

In [10]:
hyperparams = hyperparam_sets[0]

# create object to track experiment run
run = client.set_experiment_run()

# create validation split
(X_val_train, X_val_test,
 y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                             test_size=0.2,
                                                             shuffle=True)

# log hyperparameters
run.log_hyperparameters(hyperparams)
print(hyperparams, end=' ')

# create and train model
model = linear_model.LogisticRegression(**hyperparams)
model.fit(X_train, y_train)

# calculate and log validation accuracy
val_acc = model.score(X_val_test, y_val_test)
run.log_metric("val_acc", val_acc)
print("Validation accuracy: {:.4f}".format(val_acc))

# create deployment artifacts
model_api = ModelAPI(X_train, model.predict(X_train))
requirements = ["scikit-learn"]

# save and log model
run.log_model(model, model_api=model_api)
run.log_requirements(requirements)
run.log_training_data(X_train, y_train)

created new ExperimentRun: Run 558931588011935496772
{'C': 1e-06, 'solver': 'lbfgs', 'max_iter': 15} Validation accuracy: 0.7952
upload complete (custom_modules.zip)
upload complete (model.pkl)
upload complete (model_api.json)
upload complete (requirements.txt)


---

In [11]:
run.deploy(wait=True)
run

name: Run 558931588011935496772
url: https://dev.verta.ai/Convoliution/projects/8f7dc23e-3e0a-40b6-a885-d37f21ad810b/exp-runs/764fbf70-c579-4186-b8b6-9487f34b71a3
description: 
tags: []
attributes: {}
id: 764fbf70-c579-4186-b8b6-9487f34b71a3
experiment id: cdefd3f4-b54d-434f-8e02-38a3eec7ad0b
project id: 8f7dc23e-3e0a-40b6-a885-d37f21ad810b
hyperparameters: {'C': 1e-06, 'solver': 'lbfgs', 'max_iter': 15}
observations: {}
metrics: {'val_acc': 0.7951907131011609}
artifact keys: ['model.pkl', 'requirements.txt', 'custom_modules', 'model_api.json']

In [14]:
deployed_model = run.get_deployed_model()
for _, row in X_train.iterrows():
    print(deployed_model.predict([row.tolist()]))
    time.sleep(2)

[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[1]
[0]
[1]
[0]


KeyboardInterrupt: 

In [ ]:
run.undeploy()

---